
# Module 8 - Demo

Notebook ini adalah demo **pendeteksian kerutan wajah** dan **analisis usia** menggunakan YOLOv8 untuk deteksi wajah, lalu model klasifikasi untuk menentukan apakah wajah memiliki kerutan atau tidak, serta memprediksi perkiraan usia.


In [ ]:

!pip install ultralytics opencv-python matplotlib numpy deepface --quiet


In [ ]:
# -*- coding: utf-8 -*-
# Module 8: Wrinkle Detection & Age Estimation (Colab Ready)

# =============================================================
# Install optional dependencies (YOLOv8 for face detection)
# =============================================================
# (opsional, butuh internet)
# !pip install ultralytics tensorflow-macos tensorflow-metal

# =============================================================
# Imports
# =============================================================
import os, math, random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models

# =============================================================
# Config
# =============================================================
IMG_SIZE = 128
NUM_SAMPLES = 200   # kecil agar cepat
EPOCHS = 3
BATCH_SIZE = 16
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# =============================================================
# Dummy dataset generator
# =============================================================
def draw_synthetic_face(canvas, wrinkle: bool):
    h, w, _ = canvas.shape
    canvas[:] = np.random.randint(130, 180, size=canvas.shape, dtype=np.uint8)

    # wajah (oval)
    center = (w//2, h//2)
    axes = (w//3, h//2 - 10)
    face_color = (220, 200, 180)
    cv2.ellipse(canvas, center, axes, 0, 0, 360, face_color, -1)

    # mata
    eye_y = h//2 - 20
    eye_dx = 25
    cv2.circle(canvas, (center[0] - eye_dx, eye_y), 8, (30,30,30), -1)
    cv2.circle(canvas, (center[0] + eye_dx, eye_y), 8, (30,30,30), -1)

    # mulut
    cv2.ellipse(canvas, (center[0], h//2 + 25), (25, 10), 0, 0, 180, (50,50,50), 2)

    if wrinkle:
        for _ in range(np.random.randint(5, 12)):
            y = np.random.randint(h//2 - 5, h//2 + 20)
            x1 = center[0] - np.random.randint(10, axes[0] - 5)
            x2 = center[0] + np.random.randint(10, axes[0] - 5)
            cv2.line(canvas, (x1,y), (x2,y), (60,60,60), 1)
    return canvas

def generate_dummy_dataset(n=NUM_SAMPLES):
    imgs, wr, ag = [], [], []
    for _ in range(n):
        img = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
        wl = np.random.randint(0,2)
        al = np.random.randint(18,70)
        imgs.append(draw_synthetic_face(img, wl))
        wr.append(wl)
        ag.append(al)
    imgs = np.array(imgs).astype("float32")/255.0
    return imgs, np.array(wr).astype(np.float32), np.array(ag).astype(np.float32)

# =============================================================
# Build CNN multi-output
# =============================================================
def build_model():
    inputs = layers.Input((IMG_SIZE, IMG_SIZE, 3))
    x = layers.Conv2D(32,3,activation="relu",padding="same")(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64,3,activation="relu",padding="same")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128,3,activation="relu",padding="same")(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128,activation="relu")(x)

    wrinkle_out = layers.Dense(1,activation="sigmoid",name="wrinkle")(x)
    age_out = layers.Dense(1,activation="linear",name="age")(x)

    model = models.Model(inputs,[wrinkle_out, age_out])
    model.compile(optimizer="adam",
                  loss={"wrinkle":"binary_crossentropy","age":"mse"},
                  metrics={"wrinkle":"accuracy","age":"mae"})
    return model

# =============================================================
# Training
# =============================================================
X,yw,ya = generate_dummy_dataset()
split = int(0.8*len(X))
Xtr,Xva = X[:split],X[split:]
ywtr,ywva = yw[:split],yw[split:]
yatr,yava = ya[:split],ya[split:]

model = build_model()
history = model.fit(Xtr,{"wrinkle":ywtr,"age":yatr},
                    validation_data=(Xva,{"wrinkle":ywva,"age":yava}),
                    epochs=EPOCHS,batch_size=BATCH_SIZE)

# =============================================================
# Plot training curves
# =============================================================
plt.figure()
plt.plot(history.history["wrinkle_accuracy"],label="train wrinkle acc")
plt.plot(history.history["val_wrinkle_accuracy"],label="val wrinkle acc")
plt.legend(); plt.show()

plt.figure()
plt.plot(history.history["age_mae"],label="train age MAE")
plt.plot(history.history["val_age_mae"],label="val age MAE")
plt.legend(); plt.show()

# =============================================================
# Demo inference
# =============================================================
test = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)
test = draw_synthetic_face(test, wrinkle=True)
inp = (test.astype("float32")/255.0)[None,...]

pw, pa = model.predict(inp)
label = "Wrinkled" if pw[0,0]>0.5 else "No Wrinkle"
age = int(pa[0,0])
print(f"Prediction => {label}, Age {age}")

plt.imshow(cv2.cvtColor(test,cv2.COLOR_BGR2RGB))
plt.title(f"{label}, Age {age}")
plt.axis("off")
plt.show()
